In [67]:
# import libraries
import pandas as pd
import numpy as np

In [68]:
# import dataset 
df=pd.read_csv(r'C:\Users\Prabashwara Randeni\Desktop\Data Analytics\UK Data Science Job & Salary Analytics\1_CSV Load\Raw_Dataset.csv')

In [69]:
# inspec data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        750 non-null    object 
 1   Company Score  697 non-null    float64
 2   Job Title      750 non-null    object 
 3   Location       750 non-null    object 
 4   Date           750 non-null    object 
 5   Salary         635 non-null    object 
 6   Skills         742 non-null    object 
dtypes: float64(1), object(6)
memory usage: 41.1+ KB


In [70]:
#Drop Nan values
df=df.dropna()

df.isnull().sum()

Company          0
Company Score    0
Job Title        0
Location         0
Date             0
Salary           0
Skills           0
dtype: int64

Salary

In [71]:
#converting salary column to string type
df['Salary']=df['Salary'].astype('string')

#split estimation type and salary
def est(n):
    if 'Employer est.' in n:
        return 'Employer est.'
    return 'Glassdoor est.' 
    
df['Estimation Type']=df['Salary'].apply(est)


#remove unwanted items in salary column
breaker=('K','£','Employer est.','Glassdoor est.')

def b_down(n):
    for word in breaker:
        n = n.replace(word, '')
    
    n = n.replace('\xa0', '').replace('FCFA','').strip()
    n = n.replace('(','').replace(')','').replace(',','').strip()

    return n

df['Salary'] = df['Salary'].apply(b_down)

#getting annual salary rates
def salary_adjester(n):
    if "Per Hour" in n:
        n = n.replace('Per Hour','').strip()

        if "-" not in n:

            try:
                return float(n) * 40 * 52
            except ValueError:
                return None
        else:
            n = n.replace('-','').strip()
            try:
                min_salary,max_salary = map(float, n.split())
                mean_salary = (min_salary + max_salary) / 2
                return mean_salary * 40 * 52
            except ValueError:
                return None
    elif '-' in n:
        n = n.replace('-','').strip()
        try:
            min_salary, max_salary = map(float, n.split())
            mean_salary = ((min_salary + max_salary) / 2)*1000
            return mean_salary
        except ValueError:
            return None
    else:
        return float(n)*1000


df['Salary']=df['Salary'].apply(salary_adjester)

Date

In [72]:
#setup date
def cleaner(n):
    if 'd' in n:
        n = n.replace('d', '')
        if '+' in n:
            n = n.replace('+', '')
            return n
    else:
        n = n.replace('h', '1')
        return n
    return n

df['Date'] = df['Date'].apply(cleaner)


#apply random numbers to data like 30+
np.random.seed(1907)

random_picker = []

for i in range(30,45):
    random_picker.append(i)


def random_assigner(z):
    if z >=27:
        return np.random.choice(random_picker)
    else:
        return z

df['Date']=df['Date'].astype('int')

df['Date']=df['Date'].apply(random_assigner)



Location

In [73]:
#reassign City names
df['Location']=df['Location'].replace('Barking, East of England, England','London,England')
df['Location']=df['Location'].replace('Bury, North West England, England','Manchester,England')
df['Location']=df['Location'].replace('Cambridge, East of England, England','Cambridge,England')
df['Location']=df['Location'].replace('Darlington, North East England, England','Durham,England')
df['Location']=df['Location'].replace('Harwell, South East England, England','Oxford,England')
df['Location']=df['Location'].replace('Hatfield, East of England, England','Welwyn Hatfield,England')
df['Location']=df['Location'].replace('Llanishen, Monmouthshire, Wales','Cardiff,England')
df['Location']=df['Location'].replace('Luton, East of England, England','Bedfordshire,England')
df['Location']=df['Location'].replace('Preston, North West England, England','Lancashire, England')
df['Location']=df['Location'].replace('Slough, Berkshire, South East England, England','Berkshire, England')
df['Location']=df['Location'].replace('Swindon, Wiltshire, South West England, England','Wiltshire,England')
df['Location']=df['Location'].replace('Warwick, West Midlands, England','Warwickshire, England')
df['Location']=df['Location'].replace('Wellington, Herefordshire, West Midlands, England','Somerset, England')

#remove Remote from location and make new coloumn

def job_type(n):
    if 'Remote' in n:
        return 'Remote'
    else:
        return 'OnSite'
    
df['Job Type']=df['Location'].apply(job_type)

#split location to city and country

def extract_location(location):
    if 'Remote' not in location and 'United Kingdom' not in location:
        parts = location.split(",", 1)
        if len(parts) == 2:
            city = parts[0].strip()
            country = parts[1].strip()
            return city, country
        else:
            return None
    return None

df[['City', 'Country']] = df['Location'].apply(lambda loc: pd.Series(extract_location(loc)))

#drop location column
df.drop('Location',axis=1,inplace=True)

In [74]:
#export cleaned data

df.to_csv('Cleaned_Data.CSV',index=False)